In [1]:
import os

In [2]:
%pwd

'd:\\Data_science\\Projects\\chicken_Disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data_science\\Projects\\chicken_Disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Configuration for the data ingestion stage.
    """
    root_dir: Path  # Root directory where data ingestion artifacts are stored
    source_URL: str  # URL to the source data file
    local_data_file: Path  # Local path to save the downloaded data file
    unzip_dir: Path  # Directory where the data will be unzipped

In [6]:
from CNNCDC.constants import*
from CNNCDC.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from CNNCDC.utils.common import get_size
from CNNCDC import logger

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        
        else: 
            logger.info(f" File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):

        """
        Extracts a zip file into a specified directory.
        
        This method takes the path of a downloaded zip file and extracts its
        contents to the designated `unzip_dir`. It ensures the destination
        directory exists before extraction.
        """
        unzip_path = self.config.unzip_dir

        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

        logger.info(f"Zip file extracted to {unzip_path}")

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-09-25 03:37:16,652: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-25 03:37:16,655: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-25 03:37:16,657: INFO: common: created directory at: artifacts]
[2025-09-25 03:37:16,659: INFO: common: created directory at: artifacts/data_ingestion]
[2025-09-25 03:37:33,989: INFO: 3575475790: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A536:259313:263A92:4EC17D:68D46248
Accept-Ranges: bytes
Date: Wed, 24 Sep 2025 21:37:18 GMT
Via: 1.1 varnish
X-Served-By: cache-sin-wsss1830092-SIN
X-